In [1]:
import mido
from mido import MidiFile
import numpy as np
import copy

HEARING_PORT = mido.open_output()
np.set_printoptions(threshold=np.nan)

In [2]:
class Song:
    def __init__(self, name):
        self.name = name
        self.correct = True
        self.notes = np.zeros((128, 13), dtype=int)#-np.ones((128, 13))
        
        if isinstance(name, str):        
            maxnote = 0
            minnote = 127
            for msg in MidiFile(name):
                if (msg.type == "note_on" and msg.velocity > 0):
                    if msg.note < minnote:
                        minnote = msg.note
                    if msg.note > maxnote:
                        maxnote = msg.note
            if maxnote - minnote >= 13:
                print("ERROR! out of range!")
                self.correct = False
                return

            shift = 0
            if minnote < 60:
                shift = 60 - minnote
            if maxnote >= 60 + 13:
                shift = 60 + 13 - 1 - maxnote

            absolute_time_passed = 0
            for msg in MidiFile(name):
                absolute_time_passed += msg.time

                if (msg.type == "note_on" and msg.velocity > 0):
                    self.notes[round(absolute_time_passed / 0.25)][msg.note - 60 + shift] = 1
        else:
            for i, note in enumerate(name):
                if note >= 0 and note < 13:
                    self.notes[i][note] = 1
                
    def play(self):
        for msg in MidiFile(self.name).play():
            HEARING_PORT.send(msg)
            
    def transpose(self, shift):
        if (not self.correct or
           (shift > 0 and self.notes[:, -shift:].sum() != 0) or
           (shift < 0 and self.notes[:, :-shift].sum() != 0)):
            return False
        
        self.notes = np.hstack([self.notes[:, -shift:], self.notes[:, :-shift]])
        return True

In [3]:
class MySong:
    def __init__(self, played_lines=[]):
        self.notes = np.zeros((0, 13))
        
        self.mid = MidiFile()
        self.track = mido.MidiTrack()
        self.mid.tracks.append(self.track)
        
        self.time_passed = 0
        self.release = []
        
        for line in played_lines:
            self.add(line)
        
    def add(self, played):
        self.notes = np.vstack([self.notes, played])

        for i in self.release:        
            self.track.append(mido.Message('note_off', note=60+i, velocity=64, time=self.time_passed))
            self.time_passed = 0
        self.release = []

        for i, key in enumerate(played):
            if key == 1:
                self.track.append(mido.Message('note_on', note=60+i, velocity=64, time=self.time_passed))
                self.time_passed = 0
                self.release.append(i)
        self.time_passed += 256
    
    def finish(self):
        for i in self.release:        
            self.track.append(mido.Message('note_off', note=60+i, velocity=64, time=self.time_passed))
            self.time_passed = 0
        self.track.append(mido.Message('note_off', note=60, velocity=0, time=self.time_passed))
        
    def play(self):
        for msg in self.mid.play():
            HEARING_PORT.send(msg)
            
    def save_file(self, name):
        self.mid.save(name + '.mid')

# ОКИ

In [4]:
import numpy as np
from numpy import random

import matplotlib.pyplot as plt
%matplotlib inline

In [101]:
MEMORY_TIMES = 32
STARTER = 8

ADDITIONAL_MEMORY = 8

MEMORY_SIZE = MEMORY_TIMES * 13 + ADDITIONAL_MEMORY

In [102]:
NOTE_NAMES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B", "C^"]
NAMES = []
for i in range(8):
    NAMES.append("[tb" + str(i) + "]")
for t in range(-MEMORY_TIMES, 0):
    for note_name in NOTE_NAMES:
        NAMES.append(note_name + str(t))

In [242]:
TURNED = []
class Neuron:
    def __init__(self, operation, layer, ID):
        self.layer = layer
        self.ID = ID
        
        self.operation = operation
        self.inputs = []
        self.outputs = []
        self.cnt = 0
        
    def signal(self):
        self.cnt += 1
        if (self.operation == "disj" and self.cnt == 1) or (self.operation == "conj" and self.cnt == len(self.inputs)):
            TURNED.append(self.ID)
            for neuron in self.outputs:
                neuron.signal()

In [433]:
class Func:
    def __init__(self, layers, Random):
        self.layers = layers
        self.neurons = [Neuron("disj", 0, i) for i in range(MEMORY_SIZE)]
        for i in range(13):
            self.neurons.append(Neuron("disj", layers, MEMORY_SIZE + i))        
        self.Random = Random
        
    def apply(self, x):
        global TURNED
        TURNED = []
        for neuron in self.neurons:
            neuron.cnt = 0
            
        for i, val in enumerate(x):
            if val == 1:
                self.neurons[i].signal()
                
        return np.array([MEMORY_SIZE + i in TURNED for i in range(13)])
    
    def turn(self, ID, pos, verbose=False, depth=""):
        if self.neurons[ID].layer == 0:
            if verbose:
                print(depth, ID, " is input neuron, procedure failed")
            return False
        
        if (pos == 1 and self.neurons[ID].operation == "disj") or (pos == 0 and self.neurons[ID].operation == "conj"):
            if verbose:
                print(depth, "turning ", pos, ": ", ID)

            if self.Random.randint(len(self.neurons[ID].inputs) + 1) == 0:
                add_conj_reason = False
                if pos == 1:
                    element = self.neurons[TURNED[self.Random.randint(len(TURNED))]]
                    
                    if element.layer >= self.neurons[ID].layer:
                        if verbose:
                            print(depth, "layer problem, needs reason")
                        add_conj_reason = True
                
                if pos == 0 or add_conj_reason:
                    if self.neurons[ID].layer <= 1:
                        if verbose:
                            print(depth, "no place for reason :(")
                        return False
                    
                    reasonlayer = self.Random.randint(1, self.neurons[ID].layer)
                    reason = len(self.neurons)
                    reason_operation = "disj"
                    if self.neurons[ID].operation == "disj":
                        reason_operation = "conj"
                    if verbose:
                        print(depth, "create reason ", reason, " (", reasonlayer, ")")
                    
                    self.neurons.append(Neuron(reason_operation, reasonlayer, reason))
                    element = self.neurons[reason]
                
                if verbose:
                    print(depth, "new edge between layers: ", element.ID, "(", element.layer, ")", " / ", ID, "(", self.neurons[ID].layer, ")")
                element.outputs.append(self.neurons[ID])
                self.neurons[ID].inputs.append(element.ID)
                
                if add_conj_reason:
                    if not self.turn(reason, pos, verbose, depth) and verbose:
                        print(depth, "well, ok by now")
                
            else:
                element = self.neurons[self.Random.choice(self.neurons[ID].inputs)]
                if verbose:
                    print(depth, element.ID, " should be turned ", pos)

                if not self.turn(element.ID, pos, verbose, depth + "    "):
                    if verbose:
                        print(depth, "procedure failed")
                    return False
        else:
            if verbose:
                print(depth, "turning ", pos, ": ", ID)
        
            is_failed = False
            tried = False
            for input_neuron in self.neurons[ID].inputs:
                if (pos == 0 and input_neuron in TURNED) or (pos == 1 and input_neuron not in TURNED):
                    tried = True
                    if self.Random.randint(2) == 0:
                        if verbose:
                            print(depth, input_neuron, " should be turned ", pos)
                        if not self.turn(input_neuron, pos, verbose, depth + "    "):
                            if verbose:
                                print(depth, "can't turn off, procedure partly failed")
                            is_failed = True
                    else:
                        if self.neurons[ID].layer - self.neurons[input_neuron].layer == 1:
                            if verbose:
                                print(depth, "no layer for stub, procedure partly failed")
                            is_failed = True
                        else:
                            if verbose:
                                print(depth, "remove connection ", input_neuron, " / ", ID)
                            self.neurons[input_neuron].outputs.remove(self.neurons[ID])
                            self.neurons[ID].inputs.remove(input_neuron)
                            
                            stublayer = self.Random.randint(self.neurons[input_neuron].layer + 1, self.neurons[ID].layer)
                            stub = len(self.neurons)
                            stub_operation = "disj"
                            if self.neurons[ID].operation == "disj":
                                stub_operation = "conj"
                            if verbose:
                                print(depth, "create stub ", stub, " (", stublayer, ") between ", input_neuron, " (", self.neurons[input_neuron].layer, ") and ", ID, " (", self.neurons[ID].layer, ")")

                            self.neurons.append(Neuron(stub_operation, stublayer, stub))
                            self.neurons[input_neuron].outputs.append(self.neurons[stub])
                            self.neurons[stub].inputs.append(input_neuron)
                            self.neurons[stub].outputs.append(self.neurons[ID])
                            self.neurons[ID].inputs.append(stub)
                            
                            if not self.turn(stub, pos, verbose, depth + "    "):
                                if verbose:
                                    print(depth, "well, ok by now")
                                    
            if is_failed or not tried:
                if verbose:
                    if not tried:
                        print(depth, "didn't even tried to ", pos)
                    if is_failed and pos == 1:
                        print(depth, "try to add edge...")
                
                is_failed = True
                if pos == 1:
                    element = self.neurons[self.Random.randint(MEMORY_SIZE + 13, len(self.neurons))]
                    if element.layer < self.neurons[ID].layer:
                        if verbose:
                            print(depth, "new edge between layers: ", element.ID, "(", element.layer, ")", " / ", ID, "(", self.neurons[ID].layer, ")")
                        element.outputs.append(self.neurons[ID])
                        self.neurons[ID].inputs.append(element.ID)
                        is_failed = False
                        
            if is_failed:
                return False
            
        return True                            

In [434]:
class Player:
    def __init__(self):
        self.Random = np.random.RandomState(seed=179)
        self.play_note = Func(10, self.Random)
           
    def learn_to_play(self, song_to_learn, verbose=False):
        memory = np.zeros((MEMORY_SIZE), dtype=int)
        memory[-STARTER*13:] = song_to_learn.notes[:STARTER].flatten()

        t = STARTER
        memory[t % 8] = 1
        errors = 0
        
        while t < len(song_to_learn.notes):    
            output = self.play_note.apply(memory)
            
            if verbose:
                print(output.astype(int))
                       
            for note in range(13):    
                if output[note] != song_to_learn.notes[t][note]:
                    errors += 1
                    if output[note] == 0:
                        self.play_note.turn(note + MEMORY_SIZE, 1, verbose)
                    else:
                        self.play_note.turn(note + MEMORY_SIZE, 0, verbose)

            memory = np.concatenate([np.zeros((ADDITIONAL_MEMORY)), memory[ADDITIONAL_MEMORY + 13:], song_to_learn.notes[t]])
            t += 1
            memory[t % 8] = 1
            
        return errors
    
    def improvise(self, starter, length=128):
        memory = np.zeros((MEMORY_SIZE), dtype=int)
        memory[-STARTER*13:] = starter.notes[:STARTER].flatten()

        result = MySong(starter.notes[:STARTER])
        
        t = STARTER
        memory[t % 8] = 1
        while t < length:    
            output = self.play_note.apply(memory)
            result.add(output)

            memory = np.concatenate([np.zeros((ADDITIONAL_MEMORY)), memory[ADDITIONAL_MEMORY + 13:], output])
            t += 1
            memory[t % 8] = 1
        result.finish()
        
        return result

## Обучение

Создаём нового игрока и грузим пока кузнечика

In [435]:
player = Player()

In [436]:
kuznechik = Song('test/track (1).mid')

In [437]:
errors = -1
while errors != 0:
    errors = player.learn_to_play(kuznechik)
    print(errors)

76
67
55
58
48
50
62
42
53
51
52
49
52
53
50
59
55
56
65
68
64
55
65
52
53
55
58
50
60
55
48
55
55
59
46
47
55
54
54
51
55
56
52
58
65
61
61
63
50
56
53
56
51
53
71
49
52
51
45
47
49
54
54
53
51
56
52
52
58
51
57
52
55
54
56
59
58
49
50
51
56
51
58
55
49
57
63
57
58
61
55
42
40
39


KeyboardInterrupt: 

In [438]:
player.learn_to_play(kuznechik, verbose=True)

[0 0 0 0 0 0 0 0 0 0 0 0 0]
 turning  1 :  433
 1747  should be turned  1
     turning  1 :  1747
     1275  should be turned  1
         turning  1 :  1275
         3933  should be turned  1
             turning  1 :  3933
             no layer for stub, procedure partly failed
             1219  should be turned  1
                 turning  1 :  1219
                 469  should be turned  1
                     turning  1 :  469
                     254  should be turned  1
                         254  is input neuron, procedure failed
                     procedure failed
                 can't turn off, procedure partly failed
                 3045  should be turned  1
                     turning  1 :  3045
                     didn't even tried to  1
                 can't turn off, procedure partly failed
                 1588  should be turned  1
                     turning  1 :  1588
                     didn't even tried to  1
                 can't turn off, procedure par

         didn't even tried to  1
         new edge between layers:  1311 ( 4 )  /  13619 ( 5 )
     remove connection  2599  /  908
     create stub  13620  ( 7 ) between  2599  ( 5 ) and  908  ( 8 )
         turning  1 :  13620
         2599  should be turned  1
             turning  1 :  2599
             875  should be turned  1
                 turning  1 :  875
                 700  should be turned  1
                     turning  1 :  700
                     264  should be turned  1
                         264  is input neuron, procedure failed
                     procedure failed
                 procedure failed
             procedure failed
         procedure failed
     well, ok by now
     13234  should be turned  1
         turning  1 :  13234
         11025  should be turned  1
             turning  1 :  11025
             13238  should be turned  1
                 turning  1 :  13238
                 didn't even tried to  1
                 new edge between layers:  

             didn't even tried to  1
         procedure failed
     can't turn off, procedure partly failed
     no layer for stub, procedure partly failed
     no layer for stub, procedure partly failed
     no layer for stub, procedure partly failed
     3158  should be turned  1
         turning  1 :  3158
         1789  should be turned  1
             turning  1 :  1789
             no layer for stub, procedure partly failed
             remove connection  11583  /  1789
             create stub  13649  ( 2 ) between  11583  ( 1 ) and  1789  ( 3 )
                 turning  1 :  13649
                 layer problem, needs reason
                 create reason  13650  ( 1 )
                 new edge between layers:  13650 ( 1 )  /  13649 ( 2 )
                 turning  1 :  13650
                 didn't even tried to  1
                 well, ok by now
             try to add edge...
         procedure failed
     can't turn off, procedure partly failed
     3253  should be turned  

         turning  1 :  753
         2401  should be turned  1
             turning  1 :  2401
             741  should be turned  1
                 turning  1 :  741
                 remove connection  2400  /  741
                 create stub  13656  ( 2 ) between  2400  ( 1 ) and  741  ( 3 )
                     turning  1 :  13656
                     layer problem, needs reason
                     create reason  13657  ( 1 )
                     new edge between layers:  13657 ( 1 )  /  13656 ( 2 )
                     turning  1 :  13657
                     didn't even tried to  1
                     well, ok by now
                 13656  should be turned  1
                     turning  1 :  13656
                     2400  should be turned  1
                         turning  1 :  2400
                         layer problem, needs reason
                         no place for reason :(
                     procedure failed
                 can't turn off, procedure partly fa

             turning  1 :  13689
             didn't even tried to  1
             new edge between layers:  9307 ( 2 )  /  13689 ( 5 )
     12166  should be turned  1
         turning  1 :  12166
         layer problem, needs reason
         create reason  13690  ( 2 )
         new edge between layers:  13690 ( 2 )  /  12166 ( 4 )
         turning  1 :  13690
         didn't even tried to  1
         well, ok by now
     remove connection  12167  /  1677
     create stub  13691  ( 7 ) between  12167  ( 3 ) and  1677  ( 9 )
         turning  1 :  13691
         layer problem, needs reason
         create reason  13692  ( 1 )
         new edge between layers:  13692 ( 1 )  /  13691 ( 7 )
         turning  1 :  13692
         didn't even tried to  1
         well, ok by now
     12369  should be turned  1
         turning  1 :  12369
         12165  should be turned  1
             turning  1 :  12165
             12370  should be turned  1
                 turning  1 :  12370
          

             didn't even tried to  1
             new edge between layers:  13690 ( 2 )  /  13719 ( 4 )
     7159  should be turned  1
         turning  1 :  7159
         layer problem, needs reason
         create reason  13720  ( 1 )
         new edge between layers:  13720 ( 1 )  /  7159 ( 8 )
         turning  1 :  13720
         didn't even tried to  1
         well, ok by now
     7848  should be turned  1
         turning  1 :  7848
         8531  should be turned  1
             turning  1 :  8531
             no layer for stub, procedure partly failed
             12703  should be turned  1
                 turning  1 :  12703
                 12704  should be turned  1
                     turning  1 :  12704
                     remove connection  6398  /  12704
                     create stub  13721  ( 2 ) between  6398  ( 1 ) and  12704  ( 3 )
                         turning  1 :  13721
                         layer problem, needs reason
                         create

 11199  should be turned  1
     turning  1 :  11199
     remove connection  12735  /  11199
     create stub  13742  ( 6 ) between  12735  ( 5 ) and  11199  ( 9 )
         turning  1 :  13742
         12735  should be turned  1
             turning  1 :  12735
             new edge between layers:  1678 ( 3 )  /  12735 ( 5 )
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
 turning  1 :  433
 9393  should be turned  1
     turning  1 :  9393
     no layer for stub, procedure partly failed
     11417  should be turned  1
         turning  1 :  11417
         new edge between layers:  3159 ( 4 )  /  11417 ( 7 )
     11838  should be turned  1
         turning  1 :  11838
         11840  should be turned  1
             turning  1 :  11840
             didn't even tried to  1
             new edge between layers:  11188 ( 2 )  /  11840 ( 3 )
     remove connection  11912  /  9393
     create stub  13743  ( 7 ) between  11912  ( 6 ) and  9393  ( 8 )
         turning  1 :  13743
   

43

Проверим, что всё окей. Подадим на вход начало кузнечика, а дальше пусть играет сам (память занимается тем, что игрок сам нажимает на рояле). Поскольку так заданы правила, кузнечик будет воспроизведён точно

In [385]:
player.improvise(kuznechik).play()

KeyboardInterrupt: 

Вот как выглядит его булево правило:

In [14]:
player.play_note[4].print()

0 ∨ [tb2]A-18E-16A-14E-12A-10G#-8G#-6G#-2( 6.0  /  9.0 )
 ∨ [tb2][tb6]A-22E-20A-18E-16A-14E-12G#-12G#-10A-10G#-8G#-6E-4G#-2( 9.0  /  9.0 )
 ∨ [tb6]A-22E-20A-18E-16A-14G#-12G#-10G#-6E-4G#-2( 8.0  /  11.0 )
 ∨ [tb2]A-34E-32A-30E-28A-26G#-24G#-22G#-18E-16G#-14E-12G#-10A-8A-6A-2( 16.0  /  16.0 )
 ∨ [tb6]A-38E-36A-34E-32A-30G#-28G#-26G#-22E-20G#-18E-16G#-14A-12A-10A-6E-4A-2( 16.0  /  18.0 )
 ∨ [tb2]A-50E-48A-46E-44A-42G#-40G#-38G#-34E-32G#-30E-28G#-26A-24A-22A-18E-16A-14E-12A-10G#-8G#-6G#-2( 16.0  /  23.0 )
 ∨ [tb2][tb6]A-54E-52A-50E-48A-46E-44G#-44G#-42A-42G#-40G#-38E-36G#-34E-32G#-30E-28A-28G#-26A-26A-24A-22E-20A-18E-16A-14E-12G#-12G#-10A-10G#-8G#-6E-4G#-2( 17.0  /  23.0 )
 ∨ [tb6]A-54E-52A-50E-48A-46G#-44G#-42G#-38E-36G#-34E-32G#-30A-28A-26A-22E-20A-18E-16A-14G#-12G#-10G#-6E-4G#-2( 16.0  /  25.0 )



Окей, чтобы услышать что-то новое, нужно другое начало. Возьмём его из другой песенки

In [15]:
simple_song = Song('test/track (2).mid')
result = player.improvise(simple_song)

Первый шедевр:

In [ ]:
result.play()

Игрок пока использует только несколько нот. Просто в кузнечике есть не все 13 нот, так что мы можем его посмещать, чтобы получить новую информацию. Попробуем сместить кузнечика на одну ноту

In [9]:
player = Player()

In [10]:
def addAllTransposedVersions(Songs, song):
    while song.transpose(1):
        pass

    Songs.append(copy.deepcopy(song))
    while song.transpose(-1):
        Songs.append(copy.deepcopy(song))

In [11]:
Songs = []
for i in range(1, 35):
    addAllTransposedVersions(Songs, Song('test/track (' + str(i) + ').mid'))

ERROR! out of range!
ERROR! out of range!
ERROR! out of range!


In [12]:
import random
random.shuffle(Songs)

In [13]:
for song in Songs:
    errors = player.learn_to_play(song)
    print(errors)

60
81
46
42
35
25
58
33
34
23
50
56
35
42
49
41
52
48
44
32
81
42
37
59
46
44
45
45
61
54
51
0
53
72
85
83
54
35
43
56
27
42
45
22
42
57
52
51
44
61
48
21
56
31
51
0
61
41
86
48
42
55
59
44
38
52
51
55
0


In [14]:
simple_song = Song('test/whomadethis.mid')
result = player.improvise(simple_song)
result.play()

In [45]:
result.save_file("10. ")

In [15]:
simple_song = Song([0, -1, 5, -1, 4, -1, 5, -1])
result = player.improvise(simple_song)
result.play()

In [16]:
simple_song = Song([0, -1, 5, -1, 4, -1, 7, -1])
result = player.improvise(simple_song)
result.play()